In [153]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from scipy.stats import entropy

df = pd.read_csv("../raw/coursera.csv")

df = df.drop_duplicates()
df = df.drop_duplicates(subset=['Review_Text'], keep='first').reset_index(drop=True)
# df['Average_Rating'] = df['Average_rating']
# df = df.drop(columns=['Average_rating'])
# df['time_lapsed'] = df['Day_elapsed']

df


,course_name,posted_date,crawling_date,Rating,avg_rating,num_of_ratings,helpfulness,Review_Text,num_of_reviews,num_of_enrolled,num_of_top_instructor_courses,num_of_top_instructor_leaners
0,foundations-of-cybersecurity,20230729,20241130,5,4.8,27905,54,The course is well paced and they get you comf...,5510,988223,325,11153139
1,foundations-of-cybersecurity,20230521,20241130,5,4.8,27905,44,"Information was well organized, easy to learn,...",5510,988223,325,11153139
2,foundations-of-cybersecurity,20230515,20241130,5,4.8,27905,41,"For a foundation course, this one was easy to ...",5510,988223,325,11153139
3,foundations-of-cybersecurity,20240109,20241130,5,4.8,27905,32,I think this is a great start for anyone who i...,5510,988223,325,11153139
4,foundations-of-cybersecurity,20230519,20241130,5,4.8,27905,24,Surprised by the quality of this course. repea...,5510,988223,325,11153139
...,...,...,...,...,...,...,...,...,...,...,...,...
125708,introduction-to-applied-cryptography,20230905,20241130,5,4.6,66,0,This course by Professor Keith Martin is a fas...,9,9693,2,83454
125709,introduction-to-applied-cryptography,20230516,20241130,5,4.6,66,0,"Thank you so much during this 4 weeks, i gaine...",9,9693,2,83454
125710,introduction-to-applied-cryptography,20230516,20241130,5,4.6,66,0,very useful and informative course.,9,9693,2,83454
125711,introduction-to-applied-cryptography,20230903,20241130,5,4.6,66,0,Challenging and very enratainent.,9,9693,2,83454


# preprocessing_Text

In [155]:
import pandas as pd
import re

# CSV 파일 불러오기
pre_df = df.copy()

# 이모티콘만 제거하는 정규 표현식
def remove_emojis(text):
    # 이모티콘을 제외하고 영어 문장 부호는 유지
    return re.sub(r'[^\w\s.,!?\'"()\-]', '', str(text))

# 'Review_Text'와 'review_title' 열에서 이모티콘만 제거
pre_df['Review_Text'] = pre_df['Review_Text'].apply(remove_emojis)


# 영어 패턴 정의 (온점, 쉼표, 느낌표 등 허용)
english_pattern = r"^[A-Za-z0-9\s.,!?'\-\"()]+$"

# NaN 값이 있는 행 제거
pre_df = pre_df.dropna(subset=['Review_Text', 'review_title'])

# 'Review_Text'와 'review_title'이 영어로만 이루어진 행 필터링
pre_df = pre_df[
    pre_df['Review_Text'].str.match(english_pattern) & pre_df['review_title'].str.match(english_pattern)
]

# 필터링된 데이터의 행 개수 출력
print(len(pre_df))

df = pre_df

121387


# length

In [157]:
def count_words(column):
    return column.apply(lambda x: len(str(x).split()))

# 각 열의 단어 수 계산하여 새로운 열 추가
df['title_length'] = count_words(df['review_title'])
df['text_length'] = count_words(df['Review_Text'])

# 결과 출력
df

,course_name,posted_date,crawling_date,Rating,avg_rating,num_of_ratings,helpfulness,Review_Text,num_of_reviews,num_of_enrolled,num_of_top_instructor_courses,num_of_top_instructor_leaners,text_length
0,foundations-of-cybersecurity,20230729,20241130,5,4.8,27905,54,The course is well paced and they get you comf...,5510,988223,325,11153139,40
1,foundations-of-cybersecurity,20230521,20241130,5,4.8,27905,44,"Information was well organized, easy to learn,...",5510,988223,325,11153139,36
2,foundations-of-cybersecurity,20230515,20241130,5,4.8,27905,41,"For a foundation course, this one was easy to ...",5510,988223,325,11153139,38
3,foundations-of-cybersecurity,20240109,20241130,5,4.8,27905,32,I think this is a great start for anyone who i...,5510,988223,325,11153139,38
4,foundations-of-cybersecurity,20230519,20241130,5,4.8,27905,24,Surprised by the quality of this course. repea...,5510,988223,325,11153139,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125708,introduction-to-applied-cryptography,20230905,20241130,5,4.6,66,0,This course by Professor Keith Martin is a fas...,9,9693,2,83454,68
125709,introduction-to-applied-cryptography,20230516,20241130,5,4.6,66,0,"Thank you so much during this 4 weeks, i gaine...",9,9693,2,83454,27
125710,introduction-to-applied-cryptography,20230516,20241130,5,4.6,66,0,very useful and informative course.,9,9693,2,83454,5
125711,introduction-to-applied-cryptography,20230903,20241130,5,4.6,66,0,Challenging and very enratainent.,9,9693,2,83454,4


# time_lapesd

In [159]:
# Crawled_date 열 생성 및 값 채우기
df['Crawled_date'] = '24-12-05'

# Posted_Date의 형식을 변경 (05-04-21 -> 21-05-04로 변환)
df['Posted_Date'] = pd.to_datetime(df['Posted_Date'], format='%m-%d-%y', errors='coerce')
df['Crawled_date'] = pd.to_datetime(df['Crawled_date'], format='%y-%m-%d', errors='coerce')

# 날짜 차이 계산 (Crawled_date - Posted_Date)
df['time_lapsed'] = (df['Crawled_date'] - df['Posted_Date']).dt.days  # 일(day) 단위로 계산

# 결과 확인
df

,course_name,posted_date,crawling_date,Rating,avg_rating,num_of_ratings,helpfulness,Review_Text,num_of_reviews,num_of_enrolled,num_of_top_instructor_courses,num_of_top_instructor_leaners,text_length,Crawled_date,time_lapsed
0,foundations-of-cybersecurity,2023-07-29,2024-11-30,5,4.8,27905,54,The course is well paced and they get you comf...,5510,988223,325,11153139,40,24-12-05,490
1,foundations-of-cybersecurity,2023-05-21,2024-11-30,5,4.8,27905,44,"Information was well organized, easy to learn,...",5510,988223,325,11153139,36,24-12-05,559
2,foundations-of-cybersecurity,2023-05-15,2024-11-30,5,4.8,27905,41,"For a foundation course, this one was easy to ...",5510,988223,325,11153139,38,24-12-05,565
3,foundations-of-cybersecurity,2024-01-09,2024-11-30,5,4.8,27905,32,I think this is a great start for anyone who i...,5510,988223,325,11153139,38,24-12-05,326
4,foundations-of-cybersecurity,2023-05-19,2024-11-30,5,4.8,27905,24,Surprised by the quality of this course. repea...,5510,988223,325,11153139,30,24-12-05,561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125708,introduction-to-applied-cryptography,2023-09-05,2024-11-30,5,4.6,66,0,This course by Professor Keith Martin is a fas...,9,9693,2,83454,68,24-12-05,452
125709,introduction-to-applied-cryptography,2023-05-16,2024-11-30,5,4.6,66,0,"Thank you so much during this 4 weeks, i gaine...",9,9693,2,83454,27,24-12-05,564
125710,introduction-to-applied-cryptography,2023-05-16,2024-11-30,5,4.6,66,0,very useful and informative course.,9,9693,2,83454,5,24-12-05,564
125711,introduction-to-applied-cryptography,2023-09-03,2024-11-30,5,4.6,66,0,Challenging and very enratainent.,9,9693,2,83454,4,24-12-05,454


# star_dev

In [161]:
df['Deviation of star ratings'] = abs(df['Rating'] - df['Average_Rating'])
df

,course_name,posted_date,crawling_date,Rating,avg_rating,num_of_ratings,helpfulness,Review_Text,num_of_reviews,num_of_enrolled,num_of_top_instructor_courses,num_of_top_instructor_leaners,text_length,Crawled_date,time_lapsed,Deviation of star ratings
0,foundations-of-cybersecurity,2023-07-29,2024-11-30,5,4.8,27905,54,The course is well paced and they get you comf...,5510,988223,325,11153139,40,24-12-05,490,0.2
1,foundations-of-cybersecurity,2023-05-21,2024-11-30,5,4.8,27905,44,"Information was well organized, easy to learn,...",5510,988223,325,11153139,36,24-12-05,559,0.2
2,foundations-of-cybersecurity,2023-05-15,2024-11-30,5,4.8,27905,41,"For a foundation course, this one was easy to ...",5510,988223,325,11153139,38,24-12-05,565,0.2
3,foundations-of-cybersecurity,2024-01-09,2024-11-30,5,4.8,27905,32,I think this is a great start for anyone who i...,5510,988223,325,11153139,38,24-12-05,326,0.2
4,foundations-of-cybersecurity,2023-05-19,2024-11-30,5,4.8,27905,24,Surprised by the quality of this course. repea...,5510,988223,325,11153139,30,24-12-05,561,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125708,introduction-to-applied-cryptography,2023-09-05,2024-11-30,5,4.6,66,0,This course by Professor Keith Martin is a fas...,9,9693,2,83454,68,24-12-05,452,0.4
125709,introduction-to-applied-cryptography,2023-05-16,2024-11-30,5,4.6,66,0,"Thank you so much during this 4 weeks, i gaine...",9,9693,2,83454,27,24-12-05,564,0.4
125710,introduction-to-applied-cryptography,2023-05-16,2024-11-30,5,4.6,66,0,very useful and informative course.,9,9693,2,83454,5,24-12-05,564,0.4
125711,introduction-to-applied-cryptography,2023-09-03,2024-11-30,5,4.6,66,0,Challenging and very enratainent.,9,9693,2,83454,4,24-12-05,454,0.4


# readability

In [162]:
pip install textstat


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.3/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [163]:
import textstat


# 4. Readability 점수를 계산할 열 이름 설정
text_column = "Review_Text"  # CSV 파일의 텍스트가 저장된 열 이름

# 5. 새로운 열 추가 및 점수 계산
df["FOG Index"] = df[text_column].apply(lambda text: textstat.gunning_fog(str(text)))
df["Flesch Reading Ease"] = df[text_column].apply(lambda text: textstat.flesch_reading_ease(str(text)))

df

,course_name,posted_date,crawling_date,Rating,avg_rating,num_of_ratings,helpfulness,Review_Text,num_of_reviews,num_of_enrolled,num_of_top_instructor_courses,num_of_top_instructor_leaners,text_length,Crawled_date,time_lapsed,Deviation of star ratings,FOG Index,Flesch Reading Ease
0,foundations-of-cybersecurity,2023-07-29,2024-11-30,5,4.8,27905,54,The course is well paced and they get you comf...,5510,988223,325,11153139,40,24-12-05,490,0.2,11.00,85.02
1,foundations-of-cybersecurity,2023-05-21,2024-11-30,5,4.8,27905,44,"Information was well organized, easy to learn,...",5510,988223,325,11153139,36,24-12-05,559,0.2,8.11,84.98
2,foundations-of-cybersecurity,2023-05-15,2024-11-30,5,4.8,27905,41,"For a foundation course, this one was easy to ...",5510,988223,325,11153139,38,24-12-05,565,0.2,9.71,69.11
3,foundations-of-cybersecurity,2024-01-09,2024-11-30,5,4.8,27905,32,I think this is a great start for anyone who i...,5510,988223,325,11153139,38,24-12-05,326,0.2,10.76,69.11
4,foundations-of-cybersecurity,2023-05-19,2024-11-30,5,4.8,27905,24,Surprised by the quality of this course. repea...,5510,988223,325,11153139,30,24-12-05,561,0.2,14.00,56.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125708,introduction-to-applied-cryptography,2023-09-05,2024-11-30,5,4.6,66,0,This course by Professor Keith Martin is a fas...,9,9693,2,83454,68,24-12-05,452,0.4,13.27,45.76
125709,introduction-to-applied-cryptography,2023-05-16,2024-11-30,5,4.6,66,0,"Thank you so much during this 4 weeks, i gaine...",9,9693,2,83454,27,24-12-05,564,0.4,8.36,83.15
125710,introduction-to-applied-cryptography,2023-05-16,2024-11-30,5,4.6,66,0,very useful and informative course.,9,9693,2,83454,5,24-12-05,564,0.4,10.00,49.48
125711,introduction-to-applied-cryptography,2023-09-03,2024-11-30,5,4.6,66,0,Challenging and very enratainent.,9,9693,2,83454,4,24-12-05,454,0.4,11.60,50.50


# depth

In [164]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import numpy as np


# 텍스트 데이터를 전처리
df['Review_Text'] = df['Review_Text'].fillna('')

# 텍스트 데이터를 TF-IDF 벡터로 변환
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(df['Review_Text'])

# NMF 모델을 사용하여 주제 모델링 수행
num_topics = 10  # 원하는 주제 개수 설정
nmf = NMF(n_components=num_topics, random_state=42)
nmf.fit(X)

# 각 주제에 대한 단어 목록 출력
terms = np.array(vectorizer.get_feature_names_out())
for topic_idx, topic in enumerate(nmf.components_):
    print(f"Topic {topic_idx + 1}:")
    print(" ".join([terms[i] for i in topic.argsort()[:-11:-1]]))
    print()

# 주제 기여도를 0이면 아주 작은 값으로 대체
topic_contributions = nmf.transform(X)
topic_contributions = np.maximum(topic_contributions, 1e-20)  # 0을 매우 작은 값으로 대체

# Depth 계산 (p * log(p)의 합)
topic_contributions_normalized = topic_contributions / topic_contributions.sum(axis=1)[:, np.newaxis]
depth = -1 * np.sum(topic_contributions_normalized * np.log10(topic_contributions_normalized), axis=1)

# DataFrame에 depth 추가
df['depth'] = depth

df

Topic 1:
course amazing awesome beginners useful interesting recommend informative loved beginner

Topic 2:
good beginners introduction experience content start learning explanation basics learn

Topic 3:
great experience learning introduction instructor content teacher start class intro

Topic 4:
really helpful enjoyed interesting like learning assignments useful liked fun

Topic 5:
best python learn programming learning dr chuck courses basics data

Topic 6:
thank amazing dr coursera chuck google wonderful experience sir awesome

Topic 7:
excellent content teaching instructor teacher material introduction beginners way professor

Topic 8:
lot learned thanks things learnt learn new coursera helped information

Topic 9:
nice experience learning introduction explanation content teaching beginners instructor teacher

Topic 10:
easy understand follow learn way informative clear simple concepts interesting



/opt/homebrew/Cellar/jupyterlab/4.3.3/libexec/lib/python3.13/site-packages/sklearn/decomposition/_nmf.py:1742: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


,course_name,posted_date,crawling_date,Rating,avg_rating,num_of_ratings,helpfulness,Review_Text,num_of_reviews,num_of_enrolled,num_of_top_instructor_courses,num_of_top_instructor_leaners,text_length,Crawled_date,time_lapsed,Deviation of star ratings,FOG Index,Flesch Reading Ease,depth
0,foundations-of-cybersecurity,2023-07-29,2024-11-30,5,4.8,27905,54,The course is well paced and they get you comf...,5510,988223,325,11153139,40,24-12-05,490,0.2,11.00,85.02,0.615755
1,foundations-of-cybersecurity,2023-05-21,2024-11-30,5,4.8,27905,44,"Information was well organized, easy to learn,...",5510,988223,325,11153139,36,24-12-05,559,0.2,8.11,84.98,0.522035
2,foundations-of-cybersecurity,2023-05-15,2024-11-30,5,4.8,27905,41,"For a foundation course, this one was easy to ...",5510,988223,325,11153139,38,24-12-05,565,0.2,9.71,69.11,0.385761
3,foundations-of-cybersecurity,2024-01-09,2024-11-30,5,4.8,27905,32,I think this is a great start for anyone who i...,5510,988223,325,11153139,38,24-12-05,326,0.2,10.76,69.11,0.412913
4,foundations-of-cybersecurity,2023-05-19,2024-11-30,5,4.8,27905,24,Surprised by the quality of this course. repea...,5510,988223,325,11153139,30,24-12-05,561,0.2,14.00,56.25,0.675941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125708,introduction-to-applied-cryptography,2023-09-05,2024-11-30,5,4.6,66,0,This course by Professor Keith Martin is a fas...,9,9693,2,83454,68,24-12-05,452,0.4,13.27,45.76,0.749141
125709,introduction-to-applied-cryptography,2023-05-16,2024-11-30,5,4.6,66,0,"Thank you so much during this 4 weeks, i gaine...",9,9693,2,83454,27,24-12-05,564,0.4,8.36,83.15,0.646984
125710,introduction-to-applied-cryptography,2023-05-16,2024-11-30,5,4.6,66,0,very useful and informative course.,9,9693,2,83454,5,24-12-05,564,0.4,10.00,49.48,0.351222
125711,introduction-to-applied-cryptography,2023-09-03,2024-11-30,5,4.6,66,0,Challenging and very enratainent.,9,9693,2,83454,4,24-12-05,454,0.4,11.60,50.50,0.746378


# breadth

In [165]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import numpy as np

# 텍스트 데이터를 전처리
df['Review_Text'] = df['Review_Text'].fillna('')

# 텍스트 데이터를 TF-IDF 벡터로 변환
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(df['Review_Text'])

# NMF 모델을 사용하여 주제 모델링 수행
num_topics = 20  # 원하는 주제 개수 설정
nmf = NMF(n_components=num_topics, random_state=42)
nmf.fit(X)

# 각 주제에 대한 단어 목록 출력
terms = np.array(vectorizer.get_feature_names_out())
for topic_idx, topic in enumerate(nmf.components_):
    print(f"Topic {topic_idx + 1}:")
    print(" ".join([terms[i] for i in topic.argsort()[:-11:-1]]))
    print()

# 주제 기여도를 0이면 아주 작은 값으로 대체
topic_contributions = nmf.transform(X)
topic_contributions = np.maximum(topic_contributions, 1e-20)  # 0을 매우 작은 값으로 대체

# 주제 기여도를 정규화
topic_contributions_normalized = topic_contributions / topic_contributions.sum(axis=1)[:, np.newaxis]

# KL Divergence 계산 함수 (Breadth 계산)
def calculate_kl_divergence(review_topic_proportions, global_topic_proportions):
    # 0이 발생하지 않도록 작은 값 추가
    review_topic_proportions = review_topic_proportions + 1e-10
    global_topic_proportions = global_topic_proportions + 1e-10
    
    # KL 발산 계산
    kl_divergence = np.sum(review_topic_proportions * np.log10(review_topic_proportions / global_topic_proportions))
    
    # NaN 또는 Inf 값이 발생할 경우 NaN 반환
    if np.isnan(kl_divergence) or np.isinf(kl_divergence):
        return np.nan
    return kl_divergence

# 전체 문서에 대한 주제 비율의 평균 계산 (global_topic_proportions)
global_topic_proportions = np.mean(topic_contributions_normalized, axis=0)

# 각 문서에 대해 KL 발산(Breadth) 계산
breadth = np.apply_along_axis(lambda x: calculate_kl_divergence(x, global_topic_proportions), axis=1, arr=topic_contributions_normalized)

# DataFrame에 breadth 추가
df['breadth'] = breadth

df

Topic 1:
course informative recommend interesting loved beginner love wonderful like structured

Topic 2:
good content introduction start explanation instructor teaching overall teacher basics

Topic 3:
great instructor content teacher introduction start class intro information material

Topic 4:
thank coursera google sir wonderful opportunity university team knowledge professor

Topic 5:
easy understand follow informative way clear simple explained concepts content

Topic 6:
best courses coursera taken far ve online instructor teacher seen

Topic 7:
excellent content instructor teaching material teacher introduction professor way explanation

Topic 8:
lot learned things learnt thanks new ve helped information enjoyed

Topic 9:
nice content introduction explanation instructor teaching teacher experience explained way

Topic 10:
really enjoyed like liked interesting helped fun way assignments enjoy

Topic 11:
amazing instructor content teacher just experience teaching absolutely loved w

,course_name,posted_date,crawling_date,Rating,avg_rating,num_of_ratings,helpfulness,Review_Text,num_of_reviews,num_of_enrolled,num_of_top_instructor_courses,num_of_top_instructor_leaners,text_length,Crawled_date,time_lapsed,Deviation of star ratings,FOG Index,Flesch Reading Ease,depth,breadth
0,foundations-of-cybersecurity,2023-07-29,2024-11-30,5,4.8,27905,54,The course is well paced and they get you comf...,5510,988223,325,11153139,40,24-12-05,490,0.2,11.00,85.02,0.615755,0.699835
1,foundations-of-cybersecurity,2023-05-21,2024-11-30,5,4.8,27905,44,"Information was well organized, easy to learn,...",5510,988223,325,11153139,36,24-12-05,559,0.2,8.11,84.98,0.522035,0.719817
2,foundations-of-cybersecurity,2023-05-15,2024-11-30,5,4.8,27905,41,"For a foundation course, this one was easy to ...",5510,988223,325,11153139,38,24-12-05,565,0.2,9.71,69.11,0.385761,0.481265
3,foundations-of-cybersecurity,2024-01-09,2024-11-30,5,4.8,27905,32,I think this is a great start for anyone who i...,5510,988223,325,11153139,38,24-12-05,326,0.2,10.76,69.11,0.412913,0.503966
4,foundations-of-cybersecurity,2023-05-19,2024-11-30,5,4.8,27905,24,Surprised by the quality of this course. repea...,5510,988223,325,11153139,30,24-12-05,561,0.2,14.00,56.25,0.675941,0.561266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125708,introduction-to-applied-cryptography,2023-09-05,2024-11-30,5,4.6,66,0,This course by Professor Keith Martin is a fas...,9,9693,2,83454,68,24-12-05,452,0.4,13.27,45.76,0.749141,0.444210
125709,introduction-to-applied-cryptography,2023-05-16,2024-11-30,5,4.6,66,0,"Thank you so much during this 4 weeks, i gaine...",9,9693,2,83454,27,24-12-05,564,0.4,8.36,83.15,0.646984,0.742023
125710,introduction-to-applied-cryptography,2023-05-16,2024-11-30,5,4.6,66,0,very useful and informative course.,9,9693,2,83454,5,24-12-05,564,0.4,10.00,49.48,0.351222,1.115342
125711,introduction-to-applied-cryptography,2023-09-03,2024-11-30,5,4.6,66,0,Challenging and very enratainent.,9,9693,2,83454,4,24-12-05,454,0.4,11.60,50.50,0.746378,0.314778


# 파일 저장

In [166]:
df.to_csv("../preprocessed/preprocessed_coursera.csv",index = False)